In [3]:
import numpy as np
import pandas as pd
import pulp
import itertools
import folium
import openrouteservice
from geopy.distance import great_circle
from geopy.geocoders import Nominatim
from branca.element import Figure
from openrouteservice import convert
import matplotlib.pyplot as plt
import japanize_matplotlib
import sys
import time
import csv

zahyo = pd.read_csv("geo_loc_pos.csv")
idou_jikan = pd.read_csv("geo_loc_loc_time.csv")

num_places_time = len(idou_jikan)
customer_count = len(zahyo) #場所の数（id=0はdepot）
lim_day_count = 100 #何日かlim
lim_time_capacity = 60*60*6 #一日に使える時間

df_data = pd.DataFrame(zahyo.drop("地名",axis=1))

with open("geo_loc_loc_time.csv", encoding="utf_8") as f:
    reader = csv.reader(f)
    loc_loc_time = [row for row in reader]
    
with open("geo_point.csv", encoding="utf_8") as f:
    reader = csv.reader(f)
    point = [row for row in reader]

cost = [[0 for i in range(customer_count)] for j in range(customer_count)]
for i in range(num_places_time):
    cost[int(loc_loc_time[i+1][0])][int(loc_loc_time[i+1][1])] = float(loc_loc_time[i+1][2])
    cost[int(loc_loc_time[i+1][1])][int(loc_loc_time[i+1][0])] = float(loc_loc_time[i+1][2])
cost = np.array(cost)
print(cost)

visit = [[0 for i in range(customer_count)] for j in range(customer_count)]
for i in range(num_places_time):
    visit[int(loc_loc_time[i+1][0])][int(loc_loc_time[i+1][1])] = float(point[int(loc_loc_time[i+1][1])+1][1])*60
    visit[int(loc_loc_time[i+1][1])][int(loc_loc_time[i+1][0])] = float(point[int(loc_loc_time[i+1][0])+1][1])*60
visit = np.array(visit)
print(visit)



for lim_day_count in range(lim_day_count+1):
    opt_TripY = pulp.LpProblem("CVRP", pulp.LpMinimize)
    
    #変数定義
    
    X_ijk = [[[pulp.LpVariable("X%s_%s,%s"%(i,j,k), cat="Binary") if i != j else None for k in range(lim_day_count)]
            for j in range(customer_count)] for i in range(customer_count)]
    Y_ijk = [[[pulp.LpVariable("Y%s_%s,%s"%(i,j,k), cat="Binary") if i != j else None for k in range(lim_day_count)]
            for j in range(customer_count)] for i in range(customer_count)]

    #制約条件

    for j in range(1, customer_count):
        opt_TripY += pulp.lpSum(X_ijk[i][j][k] if i != j else 0 for i in range(customer_count) for k in range(lim_day_count)) == 1 

    for k in range(lim_day_count):
        opt_TripY += pulp.lpSum(X_ijk[0][j][k] for j in range(1,customer_count)) == 1
        opt_TripY += pulp.lpSum(X_ijk[i][0][k] for i in range(1,customer_count)) == 1

    for k in range(lim_day_count):
        for j in range(customer_count):
            opt_TripY += pulp.lpSum(X_ijk[i][j][k] if i != j else 0 for i in range(customer_count)) -  pulp.lpSum(X_ijk[j][i][k] for i in range(customer_count)) == 0
    for k in range(lim_day_count):
        opt_TripY += pulp.lpSum(visit[i][j] * X_ijk[i][j][k] + cost[i][j] * X_ijk[i][j][k] if i != j else 0 for i in range(customer_count) for j in range (customer_count)) <= lim_time_capacity
        
    #目的関数
    
    opt_TripY += pulp.lpSum(visit[i][j] * X_ijk[i][j][k] + cost[i][j] * X_ijk[i][j][k] if i != j else 0 for k in range(lim_day_count) for j in range(customer_count) for i in range (customer_count))
    
    #部分巡回路除去制約
    
    subtours = []
    for i in range(2,customer_count):
        subtours += itertools.combinations(range(1,customer_count), i)
    for s in subtours:
        opt_TripY += pulp.lpSum(X_ijk[i][j][k] if i !=j else 0 for i, j in itertools.permutations(s,2) for k in range(lim_day_count)) <= len(s) - 1
        
    if opt_TripY.solve() == 1:
        time_start = time.time()
        status = opt_TripY.solve()
        time_stop = time.time()
        print(f'ステータス:{pulp.LpStatus[status]}')
        print('車両数:', lim_day_count)
        print('目的関数値:',pulp.value(opt_TripY.objective))
        print('使用時間:',f"{int(pulp.value(opt_TripY.objective)//3600)}時間{int(pulp.value(opt_TripY.objective)%3600//60)}分{pulp.value(opt_TripY.objective)%3600%60:.3}秒")
        print(f'計算時間:{(time_stop - time_start):.3}(秒)')
        break
if not(opt_TripY.solve()) == 1:
    print("日数が足りません. プランを立て直してください.")
    sys.exit()
    

[[   0.  5551.  3358.8 1983.3 3763.1]
 [5551.     0.  3380.8 6439.9 2831.7]
 [3358.8 3380.8    0.  3872.4 2209.9]
 [1983.3 6439.9 3872.4    0.  5063.8]
 [3763.1 2831.7 2209.9 5063.8    0. ]]
[[   0. 3600. 3000. 2400. 1800.]
 [   0.    0. 3000. 2400. 1800.]
 [   0. 3600.    0. 2400. 1800.]
 [   0. 3600. 3000.    0. 1800.]
 [   0. 3600. 3000. 2400.    0.]]
ステータス:Optimal
車両数: 2
目的関数値: 32160.300000000003
使用時間: 8時間56分0.3秒
計算時間:0.123(秒)


In [4]:
color_list = ["red","green","purple","orange","darkred","lightred","beige","darkblue","darkgreen","cadetblue","darkpurple","white","pink","lightblue","lightgreen","gray","black","lightgray","blue"]

points_a = []
for i in range(len(zahyo)):
    points_a.append([zahyo.iloc[i,1],zahyo.iloc[i,2]])


key = "5b3ce3597851110001cf624804ffaeec7cd246038d01eb4d3a32f633"
client = openrouteservice.Client(key=key)

def route_view(points_a):
    loc_place = []
    for chimei in range(len(points_a)-1):
        p1 = points_a[chimei]
        p2 = points_a[chimei+1]
        p1r = tuple(reversed(p1))
        p2r = tuple(reversed(p2))

        # 経路計算 (Directions V2)
        routedict = client.directions((p1r, p2r),profile="foot-walking")
        geom = routedict["routes"][0]["geometry"]
        decoded = convert.decode_polyline(geom)
        for i in range(len(decoded["coordinates"])):
            loc_place.append(decoded["coordinates"][i])
    return loc_place

def reverse_lat_long(list_of_lat_long):
    return [(p[1], p[0]) for p in list_of_lat_long]


ave_lat = sum(p[0] for p in points_a)/len(points_a)
ave_lon = sum(p[1] for p in points_a)/len(points_a)
fig = Figure(width=800, height=400)

my_map = folium.Map(
    location=[ave_lat, ave_lon], 
    zoom_start=12
)

basyo_num_list = []
hiduke_judg_list = []

for k in range(lim_day_count):
    for i in range(customer_count):
        for j in range(customer_count):
            if i != j and pulp.value(X_ijk[i][j][k]) == 1:
                #print("日付：",k)
                #print("地点：",i)
                #print("目的：",j,"\n")
                basyo_num_list.append(i)
                hiduke_judg_list.append(k)
                
print(basyo_num_list,hiduke_judg_list)
day_trip_zahyo = []
hiduke_hantei = 0
bangou_1 = 0

for aaaa in hiduke_judg_list:
    bangou_2 = basyo_num_list[bangou_1]
    if not(aaaa==hiduke_hantei):
        day_trip_zahyo.append([zahyo.iloc[0,1],zahyo.iloc[0,2]])
        print(day_trip_zahyo)
        変数 = route_view(day_trip_zahyo)
        coord = reverse_lat_long(変数)
        folium.vector_layers.PolyLine(coord,
                                        color=color_list[aaaa], 
                                        weight=2.5, 
                                        opacity=1
                                        ).add_to(my_map)
        for each in range(len(day_trip_zahyo)-2):
            folium.Marker(
                    location=day_trip_zahyo[each+1],
                    icon = folium.Icon(color=color_list[aaaa])
                ).add_to(my_map)
        day_trip_zahyo = []
        day_trip_zahyo.append([zahyo.iloc[bangou_2,1],zahyo.iloc[bangou_2,2]])
        hiduke_hantei += 1
    else:
        day_trip_zahyo.append([zahyo.iloc[bangou_2,1],zahyo.iloc[bangou_2,2]])
    bangou_1 += 1
    
#最終日ルート
day_trip_zahyo.append([zahyo.iloc[0,1],zahyo.iloc[0,2]])
print(day_trip_zahyo)
変数 = route_view(day_trip_zahyo)
coord = reverse_lat_long(変数)
folium.vector_layers.PolyLine(coord,
                                color=color_list[0], 
                                weight=2.5, 
                                opacity=1
                                ).add_to(my_map)
for each in range(len(day_trip_zahyo)-2):
    folium.Marker(
            location=day_trip_zahyo[each+1],
            icon = folium.Icon(color=color_list[0])
        ).add_to(my_map)
folium.Marker(
    location=[zahyo.iloc[0,1],zahyo.iloc[0,2]],
    popup=zahyo.iloc[0,0]
).add_to(my_map)

#最後日に赤ルートならばどの順番で回っても大丈夫

my_map.save(outfile="map.html")

fig.add_child(my_map)

[0, 2, 3, 0, 1, 4] [0, 0, 0, 1, 1, 1]
[[35.0277627, 135.7727914], [34.994303, 135.7844388864188], [35.0268996, 135.7983713692728], [35.0277627, 135.7727914]]


[[35.0277627, 135.7727914], [34.967519249999995, 135.77971008109822], [34.9875598, 135.7593285404176], [35.0277627, 135.7727914]]
